In [1]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization/experiments_hoangle


In [2]:
import pandas as pd

from utils import Paths

# Read data

In [3]:
path = "../rq co2 prediction/cleaned_co2_df.csv"

raw_lunch = pd.read_csv(path, encoding='utf-8-sig', parse_dates=['Date'], low_memory=False, index_col=0)

raw_lunch.head()

,Date,Receipt time,Restaurant,Food Category,Dish,pcs,Hiilijalanjälki
0,2023-01-02,10:31,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",1.0,0.90
1,2023-01-02,10:32,600 Chemicum,Kala,Kalapuikot tillikermaviilikast,1.0,1.04
2,2023-01-02,10:32,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",1.0,0.90
3,2023-01-02,10:35,600 Chemicum,Kala,Kalapuikot tillikermaviilikast,1.0,1.04
4,2023-01-02,10:36,600 Chemicum,Liha,"Uunimakkaraa,sinappikastiketta",2.0,1.80


In [4]:
dim_dishes = pd.read_excel(Paths.dim_dishes(), index_col=None)

dim_dishes.head()

,meal_id,restaurant,category,dish
0,1,Chemicum,chicken,BBQ-Broilerikastiketta
1,2,Chemicum,chicken,Broileria appelsiini-currykastikkeessa
2,3,Chemicum,chicken,Broileria pekonikastikkeessa
3,4,Chemicum,chicken,Broileria pestokastikkeessa
4,5,Chemicum,chicken,Broilerinkoipea


# Extract CO2 amount per dish

In [5]:
lunches = (
    raw_lunch
    .replace(
        {
            '600 Chemicum': 'Chemicum',
            '610 Physicum': 'Physicum',
            '620 Exactum': 'Exactum',
            'Kala': 'fish',
            'Liha': 'meat',
            'Vegaani': 'vegan',
            'Kasvis': 'vegetarian',
            'Kana': 'chicken'
        },
    )
    .rename(columns={
        'Date': 'date',
        'Restaurant': 'restaurant',
        'Food Category': 'category',
        'Dish': 'dish',
        'Hiilijalanjälki': 'co2'
    })
    
)

lunches['pcs'] = pd.to_numeric(lunches.pcs, errors='coerce')

# Remove 'Not Napped'
lunches = lunches[lunches['category'] != 'Not Mapped']

# Remove 'takeaway'
def is_takeaway(s: str):
    return s.lower().count('take away') > 0

lunches['is_takeaway'] = lunches['dish'].map(is_takeaway)
lunches = lunches[~lunches['is_takeaway']].drop(columns='is_takeaway')

# Add dish name processing
def _f(s: str):
    s = s.split(',')[0]
    s = s.split('&')[0]
    s = s.strip()

    return s

lunches['dish'] = lunches['dish'].apply(_f)

# Extract CO2 amount
cols = [
    'restaurant',
    'category',
    'dish'
]

lunches['co2'] = lunches['co2'] / lunches['pcs']
lunches = lunches[~lunches['co2'].isna()].groupby(cols)['co2'].first().reset_index()

# Get meal_id
lunches = (
    lunches
    .merge(dim_dishes, on=['restaurant', 'category', 'dish'], how='inner')
)

lunches.head()

,restaurant,category,dish,co2,meal_id
0,Chemicum,chicken,BBQ-Broilerikastiketta,0.57,1
1,Chemicum,chicken,Broileria appelsiini-currykastikkeessa,0.64,2
2,Chemicum,chicken,Broileria pekonikastikkeessa,0.99,3
3,Chemicum,chicken,Broileria pestokastikkeessa,0.54,4
4,Chemicum,chicken,Broilerinkoipea,1.28,5


In [6]:
lunches = lunches[['meal_id', 'co2']]
lunches.to_excel(Paths.pred_co2(), index=False)